In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import tensorflow.keras as keras
from noise_generator.noisegen import NoiseHandling

In [ ]:
from audio_process.fft import stfft, normalizer, get_highest_rows, cut_fft_image, fit_image_length, expand_image_dims, train_img_generate

In [ ]:
model_velocity = keras.models.load_model('saved_model_velocity')
model_regression = keras.models.load_model('saved_model_regression')
model_categorical = keras.models.load_model('saved_model_categorical_right_align')
TRAINING_IMAGE_DIMENSIONS = (5, 1400)
NODGE_IMAGE_PIXEL_AMOUNT = 5 #Push the image to the left to adjust for incorrect position prediction
CATEGORICAL_IMAGE_CROPPED_WIDTH = TRAINING_IMAGE_DIMENSIONS[1] - 1250 #Cropped with during traing of categorical prediction model"
FFT_JUMP = 64
MEAN_TEMPO_OF_TRAINING_DATA = (18 + 25) / 2 # The mean tempo of the training data tempis 

In [ ]:
def show_image(img, width=300):
    plt.figure(figsize=(30,5))
    plt.xlim(0, width)
    plt.imshow(img)
    plt.show()

# insert an image into an empty image at start and en position
def insert_image(empty_image, image, start_pos, end_pos):
    empty_image[:, start_pos:end_pos] = image
    return empty_image

In [ ]:
ORIG_AUDIO_FILE_NAME = 'training_data/AUDIO18/ABOHLVI_03000_18_000.wav'
ORIG_AUDIO_FILE_NAME_2 = 'training_data/AUDIO18/AIAMOMCESHBBK_09900_18_000.wav'

SampleRate, signal = wavfile.read(ORIG_AUDIO_FILE_NAME_2)

length = signal.shape[0] / SampleRate

print(signal.shape)
print(SampleRate)
print('length:', length, "seconds")

Add noise to image

In [ ]:
Cnoise=NoiseHandling()
Cnoise.SetFrequencies(6000,2200,200)
Cnoise.noise_rng
signal_to_noise_ratio_db = 0 # From 0 = no noise to -15 = significant noise
# INFO:  table of SNRdb vs. digit in version\n",
# ver \t 0   1   2  3  4\n",
# SNRdB   30  10  5  2  0\n",
signal=signal/np.amax(signal)
signal_noise, someNoise= Cnoise.addNoise(signal, signal_to_noise_ratio_db)
img_noise = train_img_generate(signal_noise, FFT_JUMP)
img = train_img_generate(signal, FFT_JUMP)

show_image(img, 300)
show_image(img_noise, 300)
print(signal[:200])
print(signal_noise[:200])

Velocity prediction

In [ ]:
velocity_prediction = (model_velocity(expand_image_dims(img_noise))).numpy()[0][0] * 100
velocity_prediction

Generate new image with mean tempo of training data based on velocity prediction

In [ ]:
fft_jump_tempo_coefficient = (MEAN_TEMPO_OF_TRAINING_DATA / velocity_prediction)
rescaled_jump = FFT_JUMP * fft_jump_tempo_coefficient
img_noise_rescaled = train_img_generate(signal_noise, rescaled_jump)

Check n-px ahead and determine signal activity

In [ ]:
show_image(img_noise_rescaled)

In [ ]:
image_50_cut=img_noise_rescaled[:,:70]
image_50_cut_copy = image_50_cut.copy()
# image_50_cut_copy[:,50:100] = 0

def determine_activity(img):
    return np.sum(img, axis=1)
   
show_image(image_50_cut_copy)
sorted_row_sums = determine_activity(image_50_cut_copy)
sorted_row_sums
mean_lowest_rows = (sorted_row_sums[0] + sorted_row_sums[3] + sorted_row_sums[4]) / 3
mean_highest_rows = (sorted_row_sums[1] + sorted_row_sums[2]) / 2

diff = abs(mean_lowest_rows - mean_highest_rows)
# # return True if diff > 2 else False
sorted_row_sums, diff / (mean_highest_rows / 100)

Regression prediction to obtain first letter end position

In [ ]:
first_letter_position = model_regression(expand_image_dims(img_noise_rescaled)).numpy()[0][0] * 1400
show_image(img_noise_rescaled, width=200)
first_letter_position = first_letter_position + NODGE_IMAGE_PIXEL_AMOUNT
first_letter_position

Prepare image for categorical prediction

In [ ]:
start_position = int(CATEGORICAL_IMAGE_CROPPED_WIDTH - first_letter_position)
image_with_categorical_cropped = insert_image(
    empty_image=np.zeros(TRAINING_IMAGE_DIMENSIONS), 
    image=img_noise_rescaled[:,:int(first_letter_position)], 
    start_pos=start_position, 
    end_pos= int(first_letter_position) + start_position
)

Categorical prediction

In [ ]:

categorical_prediction = model_categorical(expand_image_dims(image_with_categorical_cropped))
show_image(image_with_categorical_cropped, width=200)
print('categorical prediction: ', np.argmax(categorical_prediction))
